# Ranking Models :

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup

In [3]:
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [4]:
import onir_pt


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# Ranking

## Antique:

### Index de Antique :

In [5]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/RI/index.zip'
extract_path = '/content/drive/MyDrive/RI/index'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Contenu du répertoire extrait : ", os.listdir(extract_path))


Contenu du répertoire extrait :  ['antique', 'trec']


In [54]:

trec_index_path = extract_path+'/antique'

index_antique = pt.IndexFactory.of(trec_index_path)

print(index_antique.getCollectionStatistics().toString())

Number of documents: 403666
Number of terms: 132468
Number of postings: 6218134
Number of fields: 1
Number of tokens: 7558887
Field names: [text]
Positions:   false



### Le dataset en question :

In [55]:
dataset_trec= pt.datasets.get_dataset('irds:antique/train/split200-valid')

In [56]:
dataset_trec

IRDSDataset('antique/train/split200-valid')

## Retrieving documents :

### Éxperimentations :

In [47]:
# Chemin vers le fichier CSV
import pandas as pd
import re
chemin_fichier = "antique_annote.csv"

# Lecture du fichier CSV

query_variations = pd.read_csv(chemin_fichier)


#### BM25 :

In [13]:
limit = 100

bm_25 = pt.BatchRetrieve(index_antique, wmodel="BM25") % limit
model_to_use = "BM25"
variante_sname = []
result_per_method = []

query_variations['qid'] = query_variations['q_id'].astype(str)
query_variations['query'] = query_variations['original_query']

for method in query_variations['method'].unique():
    print("Methode : ",method)
    query_variation = query_variations[query_variations['method'] == method]
    query_variation['query'] = query_variation['variation']

    result_per_method.append(bm_25.transform(query_variation[['qid', 'query']]))
    variante_sname.append("{}+{}".format(model_to_use, method))
    print("{}+{}".format(model_to_use, method))



Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-13-f5b8e4dc6c4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


<ipython-input-13-f5b8e4dc6c4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapQWERTY
Methode :  WordSwapRandomCharacterSubstitution


...

BM25+ramsrigouthamg/t5_paraphraser
Methode :  WordSwapEmbedding


<ipython-input-13-f5b8e4dc6c4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapEmbedding
Methode :  WordSwapWordNet


<ipython-input-13-f5b8e4dc6c4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapWordNet


In [14]:
# Experimentations :
metrics = ['ndcg_cut_10']
df = pt.Experiment(
        [bm_25] + result_per_method,
        query_variations[['qid','query']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        baseline=0,
        names = [model_to_use]+variante_sname)




[INFO] Please confirm you agree to the authors' data usage agreement found at <https://ciir.cs.umass.edu/downloads/Antique/readme.txt>
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel: s] [626kB] [3.52MB/s]


In [15]:
df

,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25,0.228598,NaN,NaN,NaN
1,BM25+WordSwapNeighboringCharacterSwap,0.155925,26.0,82.0,9.355143e-10
2,BM25+WordSwapQWERTY,0.161297,22.0,79.0,2.195817e-09
3,BM25+WordSwapRandomCharacterSubstitution,0.162307,20.0,81.0,6.247893e-09
4,BM25+naturality_by_removing_stop_words,0.226959,11.0,9.0,4.112493e-01
5,BM25+summarization_with_t5-base_from_descripti...,0.167330,25.0,78.0,2.140507e-07
6,BM25+WordInnerSwapRandom,0.228587,0.0,1.0,3.185249e-01
7,BM25+back_translation_pivot_language_de,0.161758,21.0,63.0,9.086372e-08
8,BM25+ramsrigouthamg/t5_paraphraser,0.220146,15.0,22.0,7.824204e-02
9,BM25+WordSwapEmbedding,0.175915,22.0,63.0,1.031658e-06


In [16]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM_25.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)

In [17]:

# Experimentations par requete :
metrics = ['ndcg_cut_10']
df_perq = pt.Experiment(
        [bm_25] + result_per_method,
        query_variations[['query','qid']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        perquery = True,
        names = [model_to_use]+variante_sname)

In [18]:
df_perq

,name,qid,measure,value
0,BM25,1016750,ndcg_cut_10,0.000000
1,BM25,103298,ndcg_cut_10,0.813054
2,BM25,1034050,ndcg_cut_10,0.000000
3,BM25,1037897,ndcg_cut_10,0.000000
4,BM25,1051346,ndcg_cut_10,0.119019
...,...,...,...,...
1190,BM25+summarization_with_t5-base_from_descripti...,961127,ndcg_cut_10,0.121769
1191,BM25+summarization_with_t5-base_from_descripti...,963523,ndcg_cut_10,0.000000
1192,BM25+summarization_with_t5-base_from_descripti...,98438,ndcg_cut_10,0.247396
1193,BM25+summarization_with_t5-base_from_descripti...,986260,ndcg_cut_10,0.000000


In [19]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM_25_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_perq.to_csv(file_path, index=False)

#### BM25 + RM3 :

In [ ]:
limit = 100



bm_25 = pt.BatchRetrieve(index_antique, wmodel="BM25") % limit
model = bm_25 >> pt.rewrite.RM3(index_antique) >> bm_25

model_to_use = "BM25"
variante_sname = []
result_per_method = []

query_variations['qid'] = query_variations['q_id'].astype(str)
query_variations['query'] = query_variations['original_query']

for method in query_variations['method'].unique():
    print("Methode : ",method)
    query_variation = query_variations[query_variations['method'] == method]
    query_variation['query'] = query_variation['variation']

    result_per_method.append(model.transform(query_variation[['qid', 'query']]))
    variante_sname.append("{}+{}".format(model_to_use, method))
    print("{}+{}".format(model_to_use, method))



Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-49-0455985340db>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


<ipython-input-49-0455985340db>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapQWERTY
Methode :  WordSwapRandomCharacterSubstitution


...

BM25+ramsrigouthamg/t5_paraphraser
Methode :  WordSwapWordNet


<ipython-input-49-0455985340db>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapWordNet
Methode :  WordSwapEmbedding


<ipython-input-49-0455985340db>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


BM25+WordSwapEmbedding


In [ ]:
# Experimentations :
metrics = ['ndcg_cut_10']
df = pt.Experiment(
        [model] + result_per_method,
        query_variations[['qid','query']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        baseline=0,
        names = [model_to_use]+variante_sname)
df

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] [starting] https://trec.nist.gov/data/deep/2019qrels-pass.txt
[INFO] [finished] https://trec.nist.gov/data/deep/2019qrels-pass.txt: s] [187kB] [3.37MB/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25,0.515595,NaN,NaN,NaN
1,BM25+WordSwapNeighboringCharacterSwap,0.274816,9.0,28.0,0.000064
2,BM25+WordSwapQWERTY,0.250378,5.0,33.0,0.000003
3,BM25+WordSwapRandomCharacterSubstitution,0.234741,6.0,33.0,0.000003
4,BM25+naturality_by_removing_stop_words,0.511339,1.0,3.0,0.412242
5,BM25+summarization_with_t5-base_from_descripti...,0.434432,6.0,23.0,0.015918
6,BM25+WordInnerSwapRandom,0.515595,0.0,0.0,NaN
7,BM25+back_translation_pivot_language_de,0.419520,4.0,14.0,0.028192
8,BM25+ramsrigouthamg/t5_paraphraser,0.504274,3.0,6.0,0.439185
9,BM25+WordSwapWordNet,0.365042,5.0,27.0,0.000662


In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM_25_RM3.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)

In [ ]:
# Experimentations par requete :
metrics = ['ndcg_cut_10']
df_perq = pt.Experiment(
        [model] + result_per_method,
        query_variations[['query','qid']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        perquery = True,
        names = [model_to_use]+variante_sname)

In [ ]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM_25_RM3_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_perq.to_csv(file_path, index=False)


## Méthodes  run et experiment et lancemnt des modèles de re-ranking :

In [21]:
# Chemin vers le fichier CSV
import re
chemin_fichier = "antique_annote.csv"

# Lecture du fichier CSV

query_variations = pd.read_csv(chemin_fichier)


In [22]:
limit = 100
def run(model,model_name,query_variations):
  variante_sname = []
  result_per_method = []
  #pre traitement
  query_variations['qid'] = query_variations['q_id'].astype(str)
  query_variations['query'] = query_variations['original_query']

  for method in query_variations['method'].unique():
      print("Methode : ",method)
      query_variation = query_variations[query_variations['method'] == method]
      query_variation['query'] = query_variation['variation']

      result_per_method.append(model.transform(query_variation[['qid', 'query']]))
      variante_sname.append("{}+{}".format(model_name, method))
      print("{}+{}".format(model_name, method))

  return result_per_method,variante_sname



def experiment(model,result_per_method,query_variations,dataset_trec,variante_sname,model_to_use):
  metrics = ['ndcg_cut_10']
  df = pt.Experiment(
        [model] + result_per_method,
        query_variations[['qid','query']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        baseline=0,
        names = [model_to_use]+variante_sname)
  df_perq = pt.Experiment(
        [model] + result_per_method,
        query_variations[['query','qid']].drop_duplicates(),
        dataset_trec.get_qrels(),
        metrics,
        perquery = True,
        names = [model_to_use]+variante_sname)
  return df,df_perq


#### BM25 BERT :

In [23]:

def BM25_BERT(dataset,index,limit):
      bert = onir_pt.reranker.from_checkpoint("https://macavaney.us/scibert-medmarco.tar.gz",expected_md5="854966d0b61543ffffa44cea627ab63b")
      model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> bert
      return model

BM25_et_BERT = BM25_BERT(dataset_trec,index_antique,limit)


config file not found: config


[2024-05-20 08:37:22,768][onir.util.download][DEBUG] downloaded https://macavaney.us/scibert-medmarco.tar.gz [8.24s] [499M] [130MB/s] [md5 hash verified]


[2024-05-20 08:37:49,049][onir.util.download][DEBUG] downloaded https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/pytorch_models/scibert_scivocab_uncased.tar [22.36s] [411M] [19.8MB/s] [md5 hash verified]


extracting: 411MB [1.36s, 303MB/s]
extracting: 821MB [7.94s, 103MB/s] 


In [24]:
result_per_method,variante_sname = run(BM25_et_BERT,"BM25+BERT",query_variations)
df,df_q = experiment(BM25_et_BERT,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+BERT")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-22-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']
[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/684f7015aff377062a758e478476aac8
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt
docs_iter:   0%|                                    | 0/403666 s<?, ?doc/s]
https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt: 0.0%| 0.00/93.6M s<?, ?B/s]
https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt: 0.2%| 156k/93.6M s<01:19, 1.18MB/s]
h

[2024-05-20 08:38:26,139][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 08:38:26,669][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4678 s<?, ?it/s]

[2024-05-20 08:40:59,605][onir_pt][DEBUG] [finished] batches: [02:33] [4678it] [30.59it/s]
BM25+BERT+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

batches:   0%|          | 0/4857 s<?, ?it/s]

[2024-05-20 09:08:19,408][onir_pt][DEBUG] [finished] batches: [02:44] [4857it] [29.63it/s]
[2024-05-20 09:08:24,917][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 09:08:24,921][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2024-05-20 09:11:12,196][onir_pt][DEBUG] [finished] batches: [02:47] [4857it] [29.04it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+BERT,0.298741,NaN,NaN,NaN
1,BM25+BERT+WordSwapNeighboringCharacterSwap,0.219756,42.0,87.0,1.095150e-08
2,BM25+BERT+WordSwapQWERTY,0.228915,31.0,85.0,5.281410e-09
3,BM25+BERT+WordSwapRandomCharacterSubstitution,0.218040,24.0,100.0,6.743537e-11
4,BM25+BERT+naturality_by_removing_stop_words,0.278089,39.0,75.0,3.506904e-04
5,BM25+BERT+summarization_with_t5-base_from_desc...,0.213728,31.0,107.0,5.680351e-12
6,BM25+BERT+WordInnerSwapRandom,0.296807,46.0,46.0,5.044115e-01
7,BM25+BERT+back_translation_pivot_language_de,0.232330,35.0,79.0,5.068745e-07
8,BM25+BERT+ramsrigouthamg/t5_paraphraser,0.287625,32.0,56.0,2.060671e-02
9,BM25+BERT+WordSwapEmbedding,0.246034,36.0,75.0,1.201366e-07


In [27]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_BERT.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_BERT_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

#### BM25 KNRM:

In [28]:
def BM25_KNRM(dataset,index,limit):
      knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz',  expected_md5="d70b1d4f899690dae51161537e69ed5a")
      model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> knrm
      return model
model_bm25_knrm =  BM25_KNRM (dataset_trec,index_antique,limit)

[2024-05-20 09:16:08,410][onir.util.download][DEBUG] downloaded https://macavaney.us/knrm.medmarco.tar.gz s] [1.43k] [1.11MB/s] [md5 hash verified]
[2024-05-20 09:16:08,431][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2024-05-20 09:16:25,783][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2024-05-20 09:16:25,788][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [16.89s] [682M] [48.2MB/s]
[2024-05-20 09:16:25,805][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [17.38s]
[2024-05-20 09:16:25,805][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2024-05-20 09:16:47,540][WordvecHashVocab][DEBUG] [finished] extracting vecs [21.73s]
[2024-05-20 09:16:47,543][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2024-05-20 09:19:28,643][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:41]
[2024-05-20 09:19:28,896][WordvecHashVocab][DEBUG] [starting] writ

In [29]:
result_per_method,variante_sname = run(model_bm25_knrm,"BM25+KNRM",query_variations)
df,df_q = experiment(model_bm25_knrm,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+KNRM")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-22-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-20 09:20:05,420][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 09:20:05,761][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4678 s<?, ?it/s]

[2024-05-20 09:20:13,615][onir_pt][DEBUG] [finished] batches: [7.85s] [4678it] [595.67it/s]
BM25+KNRM+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

batches:   0%|          | 0/4857 s<?, ?it/s]

[2024-05-20 09:22:20,220][onir_pt][DEBUG] [finished] batches: [8.15s] [4857it] [595.64it/s]
[2024-05-20 09:22:25,854][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 09:22:25,856][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2024-05-20 09:22:34,021][onir_pt][DEBUG] [finished] batches: [8.16s] [4857it] [594.86it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+KNRM,0.195348,NaN,NaN,NaN
1,BM25+KNRM+WordSwapNeighboringCharacterSwap,0.149475,35.0,66.0,4.519424e-06
2,BM25+KNRM+WordSwapQWERTY,0.153281,32.0,70.0,3.504599e-05
3,BM25+KNRM+WordSwapRandomCharacterSubstitution,0.150131,28.0,76.0,1.261441e-06
4,BM25+KNRM+naturality_by_removing_stop_words,0.225525,79.0,40.0,3.297044e-04
5,BM25+KNRM+summarization_with_t5-base_from_desc...,0.172450,56.0,75.0,9.192195e-02
6,BM25+KNRM+WordInnerSwapRandom,0.195337,1.0,1.0,7.250729e-01
7,BM25+KNRM+back_translation_pivot_language_de,0.135938,16.0,75.0,1.014742e-08
8,BM25+KNRM+ramsrigouthamg/t5_paraphraser,0.188036,41.0,37.0,2.099305e-01
9,BM25+KNRM+WordSwapEmbedding,0.172996,39.0,58.0,2.184511e-03


In [30]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_KNRM.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_KNRM_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

#### BM25_T5:

In [31]:
from pyterrier_t5 import MonoT5ReRanker

def BM25T5 (dataset,index,limit):
      monoT5 = MonoT5ReRanker()
      model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> monoT5
      return model
model_bm25_T5 =  BM25T5 (dataset_trec,index_antique,limit)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k s<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M s<?, ?B/s]

config.json:   0%|          | 0.00/1.21k s<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.84k s<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M s<?, ?B/s]

In [32]:
result_per_method,variante_sname = run(model_bm25_T5,"BM25+T5",query_variations)
df,df_q = experiment(model_bm25_T5,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+T5")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-22-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


monoT5:   0%|          | 0/4678 s<?, ?batches/s]

BM25+T5+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


<ipython-input-22-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


...

monoT5:   0%|          | 0/5001 s<?, ?batches/s]

BM25+T5+WordSwapWordNet


monoT5:   0%|          | 0/4857 s<?, ?batches/s]

monoT5:   0%|          | 0/4857 s<?, ?batches/s]

,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+T5,0.333343,NaN,NaN,NaN
1,BM25+T5+WordSwapNeighboringCharacterSwap,0.250900,32.0,93.0,1.152331e-09
2,BM25+T5+WordSwapQWERTY,0.265228,35.0,82.0,1.618975e-08
3,BM25+T5+WordSwapRandomCharacterSubstitution,0.245766,32.0,94.0,8.339385e-12
4,BM25+T5+naturality_by_removing_stop_words,0.319982,54.0,70.0,3.146140e-02
5,BM25+T5+summarization_with_t5-base_from_descri...,0.239296,35.0,111.0,7.875166e-14
6,BM25+T5+WordInnerSwapRandom,0.325507,33.0,62.0,1.250575e-02
7,BM25+T5+back_translation_pivot_language_de,0.258100,29.0,82.0,8.703912e-08
8,BM25+T5+ramsrigouthamg/t5_paraphraser,0.321404,28.0,52.0,3.644629e-02
9,BM25+T5+WordSwapEmbedding,0.281352,30.0,77.0,2.247966e-09


In [33]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_T5.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_T5_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

####  BM25 EPIC:

In [34]:

def EPIC(dataset,index,limit):
  config = {'batch_size': 4}
  reranker = onir_pt.reranker.from_checkpoint("https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz", config=config)
  model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> reranker
  return model

model_EPIC = EPIC(dataset_trec,index_antique,limit)


[2024-05-20 10:08:54,797][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz; consider adding expected_md5="c184a924debf63b5115e34a3b92377e0" to ensure data integrity.
[2024-05-20 10:08:54,807][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz [7.17s] [495M] [124MB/s]


100%|██████████| 231508/231508 s<0ms, 2723164.01B/s]
100%|██████████| 433/433 s<0ms, 568607.90B/s]
100%|██████████| 440473133/440473133 [10.43s<0ms, 42218270.36B/s]  


In [35]:
result_per_method,variante_sname = run(model_EPIC,"BM25+EPIC",query_variations)
df ,df_q = experiment(model_EPIC,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+EPIC")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-22-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-20 10:11:01,986][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 10:11:02,126][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4678 s<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/onir/rankers/epic.py:107: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  result = torch.sparse.FloatTensor(torch.stack((idx0, idx1)), tok_salience.flatten(), s)


...

batches:   0%|          | 0/4857 s<?, ?it/s]

[2024-05-20 10:59:19,449][onir_pt][DEBUG] [finished] batches: [04:29] [4857it] [18.02it/s]
[2024-05-20 10:59:25,100][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 10:59:25,104][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/4857 s<?, ?it/s]

[2024-05-20 11:03:55,634][onir_pt][DEBUG] [finished] batches: [04:31] [4857it] [17.95it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+EPIC,0.265964,NaN,NaN,NaN
1,BM25+EPIC+WordSwapNeighboringCharacterSwap,0.184004,32.0,92.0,2.128838e-10
2,BM25+EPIC+WordSwapQWERTY,0.192009,26.0,92.0,8.363589e-10
3,BM25+EPIC+WordSwapRandomCharacterSubstitution,0.187947,27.0,92.0,2.297166e-11
4,BM25+EPIC+naturality_by_removing_stop_words,0.269284,69.0,46.0,5.079424e-01
5,BM25+EPIC+summarization_with_t5-base_from_desc...,0.200639,43.0,89.0,4.712829e-08
6,BM25+EPIC+WordInnerSwapRandom,0.266102,2.0,6.0,6.763371e-01
7,BM25+EPIC+back_translation_pivot_language_de,0.203180,34.0,71.0,1.601625e-06
8,BM25+EPIC+ramsrigouthamg/t5_paraphraser,0.261390,40.0,39.0,3.648061e-01
9,BM25+EPIC+WordSwapEmbedding,0.213856,34.0,84.0,7.183161e-08


In [36]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_EPIC.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_EPIC_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)

#### BM25 CKNRM :

In [37]:

def CKNRM(dataset,index,limit):
  reranker = onir_pt.reranker.from_checkpoint("https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz")
  model = pt.BatchRetrieve(index, wmodel="BM25") % limit >> pt.text.get_text(dataset, 'text') >> reranker
  return model

model_CKNRM = CKNRM(dataset_trec,index_antique,limit)


[2024-05-20 11:05:00,035][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz; consider adding expected_md5="6e57b8d728151debcb23479c8a41f4b0" to ensure data integrity.
[2024-05-20 11:05:00,043][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz s] [860k] [9.51MB/s]
[2024-05-20 11:05:00,063][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2024-05-20 11:05:14,776][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [14.71s]


In [38]:
result_per_method,variante_sname = run(model_CKNRM,"BM25+CKNRM",query_variations)
df,df_q = experiment(model_CKNRM,result_per_method,query_variations,dataset_trec,variante_sname,"BM25+CKNRM")
df

Methode :  WordSwapNeighboringCharacterSwap


<ipython-input-22-831b6c94f792>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_variation['query'] = query_variation['variation']


[2024-05-20 11:05:32,244][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 11:05:32,646][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/2339 s<?, ?it/s]

[2024-05-20 11:06:40,739][onir_pt][DEBUG] [finished] batches: [01:08] [2339it] [34.35it/s]
BM25+CKNRM+WordSwapNeighboringCharacterSwap
Methode :  WordSwapQWERTY


...

batches:   0%|          | 0/2429 s<?, ?it/s]

[2024-05-20 11:18:59,169][onir_pt][DEBUG] [finished] batches: [01:10] [2429it] [34.53it/s]
[2024-05-20 11:19:04,728][onir_pt][DEBUG] using GPU (deterministic)
[2024-05-20 11:19:04,730][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/2429 s<?, ?it/s]

[2024-05-20 11:20:14,869][onir_pt][DEBUG] [finished] batches: [01:10] [2429it] [34.63it/s]


,name,ndcg_cut_10,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BM25+CKNRM,0.206472,NaN,NaN,NaN
1,BM25+CKNRM+WordSwapNeighboringCharacterSwap,0.144357,25.0,80.0,5.697300e-09
2,BM25+CKNRM+WordSwapQWERTY,0.155270,23.0,67.0,3.080402e-06
3,BM25+CKNRM+WordSwapRandomCharacterSubstitution,0.147622,23.0,80.0,5.002043e-08
4,BM25+CKNRM+naturality_by_removing_stop_words,0.215285,68.0,50.0,2.657696e-01
5,BM25+CKNRM+summarization_with_t5-base_from_des...,0.167238,50.0,77.0,1.329793e-03
6,BM25+CKNRM+WordInnerSwapRandom,0.197762,28.0,46.0,2.980473e-02
7,BM25+CKNRM+back_translation_pivot_language_de,0.143829,19.0,68.0,6.133165e-10
8,BM25+CKNRM+ramsrigouthamg/t5_paraphraser,0.195704,30.0,37.0,8.307576e-02
9,BM25+CKNRM+WordSwapEmbedding,0.168927,25.0,65.0,1.790301e-06


In [39]:
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_CKNRM.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df.to_csv(file_path, index=False)
# Chemin où vous souhaitez enregistrer le fichier CSV dans Google Drive
file_path = '/content/drive/My Drive/RI/antique_BM25_CKNRM_q.csv'

# Enregistrer le DataFrame en tant que fichier CSV
df_q.to_csv(file_path, index=False)